### Importing required libraries

In [1]:
import torch
from torch.utils.data import Dataset
import torchvision #provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
import torch.nn as nn

In [2]:
mnist_train_set = torchvision.datasets.MNIST(
            root='./try1',
            train=True,
            download=True,
            transform=transforms.Compose([
                transforms.ToTensor()
            ])
        )

100.0%


Extracting ./try1\MNIST\raw\train-images-idx3-ubyte.gz to ./try1\MNIST\raw



100.0%


Extracting ./try1\MNIST\raw\train-labels-idx1-ubyte.gz to ./try1\MNIST\raw



100.0%


Extracting ./try1\MNIST\raw\t10k-images-idx3-ubyte.gz to ./try1\MNIST\raw



100.0%

Extracting ./try1\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./try1\MNIST\raw



In [33]:
# trying the sum of 60000 random integers with labels of train dataset of mnist
sum_labels = mnist_train_set.targets + torch.randint(0, 9, (1, 60000)).squeeze()
sum_labels
# torch.stack((mnist_train_set.targets, sum_labels), dim=1)

tensor([ 8,  1,  9,  ...,  5, 11, 13])

In [31]:
type(mnist_train_set[0])

tuple

In [32]:
len(mnist_train_set.data)

60000

In [35]:
rn = torch.randint(0, 9, (1, 60000)).squeeze()
rn[1]

tensor(3)

In [76]:
class MNISTWithNumbers(Dataset):
    def __init__(self):
        self.mnist_data = torchvision.datasets.MNIST(
            root='./data',
            train=True,
            download=True,
            transform=transforms.Compose([
                transforms.ToTensor()
            ])
        )
        self.random_numbers = torch.randint(0, 9, (1, 60000)).squeeze()

    def __getitem__(self, index):
        image = self.mnist_data.data[index].unsqueeze(0)
        label = self.mnist_data.targets[index]
        number = self.random_numbers[index].unsqueeze(0)
        return image, label, number

    def __len__(self):
        return len(self.mnist_data.data)
    
    @property
    def train_labels(self):
        sum_labels = self.mnist_data.targets + self.random_numbers
        return self.mnist_data.targets, sum_labels


In [77]:
mnist_with_numbers = MNISTWithNumbers()

In [78]:
targets, sum_labels = mnist_with_numbers.train_labels
targets.shape, sum_labels.shape

(torch.Size([60000]), torch.Size([60000]))

In [79]:
sample = next(iter(mnist_with_numbers))
image, label, number = sample
image.shape, label.shape, number.shape

(torch.Size([1, 28, 28]), torch.Size([]), torch.Size([1]))

In [80]:
train_data_loader = torch.utils.data.DataLoader(
    mnist_with_numbers,
    batch_size=32,
    shuffle=True
)

In [82]:
samples = next(iter(train_data_loader))
images, labels, numbers = samples
images.shape, labels.shape, numbers.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]), torch.Size([32, 1]))